### Свой профайлер

гугл диск говорит, что ресурс нельзя забрать, поэтому взял clothing-dataset с гитхаба переделал под нужный формат - папка дата и внутри папка train

также почему-то 5 картинок не было поэтому из csv удалил их

```
d028580f-9a98-4fb5-a6c9-5dc362ad3f09
1d0129a1-f29a-4a3f-b103-f651176183eb
784d67d4-b95e-4abb-baf7-8024f18dc3c8
c60e486d-10ed-4f64-abab-5bb698c736dd
040d73b7-21b5-4cf2-84fc-e1a80231b202
```

далее написал профайлер, каждый шаг обновляю дикт в котором храню куда события для pre forward, post forward, pre backward, post backward (только если сейчас не wait шаг)

далее когда доходит до profiler.step() закидываю все эти события в tmp список и обновляю дикт

периодически проверяю что событие из tmp завершилось и перекидываю его тогда в основной список

во время to_perfetto проверяю что нет незавершенных события и кидаю ворнинг если есть (надо sync сделать)

получились следующие резы в perfetto:

на трейне

<img src="image_train.jpg" width="1000">

на валидации

<img src="image_val.jpg" width="1000">


тем временем у профайлера торча

на трейне

<img src="image_train_torch.jpg" width="1000">

на валидации

<img src="image_val_torch.jpg" width="1000">


на валидации видим тоже самое но у торча более подробно, но вот на трейне у них весь backward засунут в один блок

также у них есть на cpu профайлер поэтому можно легко увидеть основную проблему с данным обучением

<img src="image_val_torch2.jpg" width="1000">

как видно например на валидации, обучение занимает супер мало времени и почти все время уходит на даталоадер, но про это во второй части

### Проблемы

1. у нас как я показал на прошлом скрине ботлнек в даталоадере, при этом он ранится в один поток, поставил 64

также добавил чтобы мы сразу считали в торче и не было оверхед на перевод из PIL в torch (там если посмотреть PIL считывание занимало где-то 1/4 от одного getitem, но хз засчитывается ли это за баг)

стало выглядить вот так

<img src="train_log_bug1.jpg" width="1000">

2. чтобы он сразу перекидывал в нужную память добавляем pin memory в даталоадер



<img src="train_log_bug2.jpg" width="1000">

3. перед форвардом у нас есть куда синхронайз нужно добавить non blocking в .to()

<img src="train_log_bug33.jpg" width="1000">

4. прошлое улучшение убрало синк но не ускорило тк у нас есть еще один

также между форвардом и беквардом у нас есть синхронизация, оставляем detach() но убираем .item() и делаем его в конце эпохи

<img src="train_log_bug4.jpg" width="1000">

5. ну тут без профайлера но наша валидация делается с подсчемтом градиента так что добавляем with torch.no_grad()

6. и еще одна проблема тоже не связанная с профайлером, но мы считаем все в fp32 так что добавляем скейлер

в итоге профайлер показывает вот такую картинку

<img src="train_log_bug6.jpg" width="1000">